# Ensemble Learning - 1
***
이번 챕터에서는 앙상블 학습에 대해서 알아보도록 하겠습니다. '백지장도 맞들면 낫다.'라는 말 많이들 들어보셨을텐데요. 앙상블을 설명하기에 아주 알맞은 말이라고 생각됩니다. 앙상블은 여러 개의 분류기 혹은 모델들의 예측 값들을 결합하여 최종적으로 좀 더 좋은 성능을 내는 기법입니다. '집단지성'도 어찌보면 앙상블과 일맥상통한다고 볼 수 있겠습니다. <br>
정형 데이터의 문제 해결에 앙상블 알고리즘이 강세를 보이고 있는데 그 중에서도 부스팅 계열의 알고리즘(xgboost, lightgbm 등)이 뛰어난 성능을 보여주고 있습니다. 이번 챕터에서는 전통적인 앙상블의 유형인 보팅(Voting), 배깅(Bagging), 부스팅(Boosting), 스태킹(Stacking)에 대해서 알아보도록 하겠습니다.<br>

- 보팅(Voting) : 서로 다른 알고리즘을 활용하여 예측한 결과값을 바탕으로 최종예측
- 부스팅(Boosting) : 예측이 틀린 데이터는 다른 분류기가 학습시 가중치 부여하여 학습 및 예측 진행

- 스태킹(Stacking) : 서로 다른 모델의 예측 결과값을 다시 학습데이터로 사용하여 다른 모델로 재학습 및 예측 진행

- 배깅(Bootstrap Aggregating의 줄임말) : train 데이터에서 중복을 허용하여 샘플링하는 방식을 통해 같은 분류기내에서 서로 다른 데이터로 학습

     cf) 페이스팅(Pasting) : 중복을 허용하지 않고 샘플링
<br>
***
    
먼저 voting classifier부터 살펴보도록 하겠습니다. 데이터는 사이킷런에 내장되어 있는 위스콘신 유방암 데이터를 활용하도록 하겠습니다.

In [57]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

cancer = load_breast_cancer()

df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


데이터를 구성한 후 '로지스틱회귀'와 'KNN'을 기반으로 '소프트 보팅'방식을 활용해 새로운 모델을 만들어보도록 하겠습니다.

In [58]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size = .2, random_state = 1212)

In [59]:
log_clf = LogisticRegression() # 로지스틱 회귀 모델
knn_clf = KNeighborsClassifier() # KNN 모델

In [60]:
log_clf.fit(X_train, y_train) # 로지스틱 회귀 모델 학습
knn_clf.fit(X_train, y_train) # KNN 모델 학습

KNeighborsClassifier()

In [61]:
log_pred = log_clf.predict(X_test) # 로지스틱 회귀 모델로 예측
knn_pred = knn_clf.predict(X_test) # KNN 모델로 예측

In [62]:
vot_clf = VotingClassifier(estimators = [('log', log_clf), ('knn', knn_clf)], voting = 'soft')

Voting 방식에는 크게 'hard' 방식과 'soft' 방식이 있습니다. 먼저 'hard'는 '다수결' 원칙이라고 할 수 있겠는데요.<br> 실제 예측한 'class' 값들을 기준으로 많이 분류된 'class'를 최종 예측값으로 결정하게 됩니다.<br> 반면 'soft' 방식은 'class'별 예측 결과 확률 값을 기준으로 분류기들의 확률값을 평균을 내어 이 중 가장 높은 확률을 가진 'class'를 최종 결과 값으로 선정하게 됩니다. 

In [63]:
vot_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('log', LogisticRegression()),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [64]:
vot_pred = vot_clf.predict(X_test)

In [78]:
print(f'{log_clf.__class__.__name__}모델의 정확도는 {np.round(accuracy_score(y_test, log_pred), 6) * 100}%')
print(f'{knn_clf.__class__.__name__}모델의 정확도는 {np.round(accuracy_score(y_test, knn_pred), 4) * 100}%')
print(f'{vot_clf.__class__.__name__}모델의 정확도는 {np.round(accuracy_score(y_test, vot_pred), 4) * 100}%')

LogisticRegression모델의 정확도는 94.7368%
KNeighborsClassifier모델의 정확도는 95.61%
VotingClassifier모델의 정확도는 97.37%


LogisticRegression의 정확도는 94.74%를 기록했고 KNN 모델의 정확도는 95.61%를 기록했습니다.<br>
Voting 방식의 경우는 훨씬 높은 97.37%의 정확도를 보였습니다.<br>
항상 Voting 방식의 모델이 성능이 우수하다고는 장담할 수 없습니다. 데이터의 형태와 분포 및 관련Task 등 다양한 요건들을 고려하여 여러 시도를 해보는 과정중의 일부로 접근해야 할 것입니다.
***
이번에는 RandomForest 알고리즘에 대해 알아보도록 하겠습니다. 앞서 배깅은 같은 알고리즘 내에서 데이터를 중복 허용의 샘플링을 통해 학습하는 것이라고 했습니다. 배깅의 대표적인 알고리즘으로는 RandomForest가 있는데 Chapter2에서 살펴본 결정트리 알고리즘을 기반으로 구성되어있습니다. 여러개의 결정트리 분류기를 중복 허용된 데이터 세트를 활용하는 알고리즘이 RandomForest라고 할 수 있겠습니다. 바로 실습과 함께 더 알아보도록 하죠.

In [66]:
from sklearn.ensemble import RandomForestClassifier

In [67]:
rf_clf = RandomForestClassifier(random_state = 1212)

In [68]:
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=1212)

In [69]:
rf_pred = rf_clf.predict(X_test)

In [79]:
print(f'{rf_clf.__class__.__name__}모델의 정확도는 {np.round(accuracy_score(y_test, rf_pred), 4) * 100}%')

RandomForestClassifier모델의 정확도는 98.25%


위에서 사용한 LogisticRegression과 KNN 모델 보다 훨씬 더 높은 정확도를 기록했습니다. 심지어 두 모델을 Voting한 모델의 정확도 보다도 더 좋은 것을 알 수 있습니다.
***
이번에는 RandomForest의 하이퍼 파라미터를 살펴보겠습니다. 크게 아래의 파라미터들을 중요하다고 볼 수 있습니다.

- n_estimatros : 결정 트리 개수 / default = 10 / 높을수록 성능이 높아지나 과적합 및 오랜 수행 시간 소요

- max_depth : 결정트리와 마찬가지로 트리의 깊이

- min_samples_leaf, min_samples_split 등등
***
사이킷런의 GridSearchCV를 통해 하이퍼 파라미터 탐색을 해보도록 하겠습니다.

In [80]:
from sklearn.model_selection import GridSearchCV

In [81]:
param = {'n_estimators' : [30, 50 ,150, 300, 500], 'max_depth' : [3,4,5,6,7], 'min_samples_leaf' : [2,4,8]}

In [82]:
rf_clf = RandomForestClassifier(random_state = 1212, n_jobs = -1)
grid_rf = GridSearchCV(rf_clf, param, cv = 4, n_jobs = -1)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=4,
             estimator=RandomForestClassifier(n_jobs=-1, random_state=1212),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [2, 4, 8],
                         'n_estimators': [30, 50, 150, 300, 500]})

In [85]:
print('최종 하이퍼 파라미터는', grid_rf.best_params_)
print(f'최고 정확도는 {np.round(grid_rf.best_score_, 4) * 100}%')

최종 하이퍼 파라미터는 {'max_depth': 7, 'min_samples_leaf': 2, 'n_estimators': 500}
최고 정확도는 96.48%


In [89]:
final_rf_clf = RandomForestClassifier(**grid_rf.best_params_, random_state = 1212)
final_rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_leaf=2, n_estimators=500,
                       random_state=1212)

In [90]:
final_rf_pred = final_rf_clf.predict(X_test)

In [91]:
print(f'{final_rf_clf.__class__.__name__}모델의 정확도는 {np.round(accuracy_score(y_test, final_rf_pred), 4) * 100}%')

RandomForestClassifier모델의 정확도는 96.49%


하이퍼 파라미터 튜닝을 실행했지만 튜닝 전 모델의 정확도가 더 높았습니다. 충분한 컴퓨팅 자원과 시간이 있다면 더 많은 파라미터 조합을 탐색하여 보다 좋은 성능을 기록할 수 있을 것입니다.<br>

지금까지 앙상블 기법 중 하나인 RandomForest에 대해서 알아보았는데요. 다음 챕터에서는 부스팅 계열의 알고리즘에 대해서 알아보도록 하겠습니다. 감사합니다.